In [1]:
#import required libraries

import numpy as np 
import imutils
import cv2


In [2]:
# this function sort the contours in differet order
# takes two argument 1.contours, 2.method
# return two parameter 1.sorted contour 2.bounding box

def sort_contours(cnts, method="left-to-right" ):
    reverse= False
    i= 0

    if method== 'right-to-left' or method== ' bottom-to-top':
        reverse= True
    
    if method=='top-to-bottom' or method== 'bottom-to-top':
        i= 1

    boundingboxes= [cv2.boundingRect(c) for c in cnts]
    (cnts, boundingboxes)= zip(*sorted(zip(cnts, boundingboxes), key= lambda b:b[1][i], reverse= reverse))
    return (cnts, boundingboxes)

In [3]:
# function to draw contour numbers 

def draw_contour(image, c, i):
    m = cv2.moments(c)
    cx= int(m['m10']/ m['m00'])
    cy= int(m['m01']/ m['m00'])

    cv2.putText(image, '#{}'.format(i+1), (cx-20, cy), cv2.FONT_HERSHEY_COMPLEX, 1.0, (255,255,255), 2)

    return image


In [4]:
# read image
image= cv2.imread('dataset/image_01.png')
accumEdged= np.zeros(image.shape[:2], dtype='uint8')
for chan in cv2.split(image):
    chan = cv2.medianBlur(chan, 11)
    edged= cv2.Canny(chan, 50, 200)
    accumEdged= cv2.bitwise_or(accumEdged, edged)

# display image having countours
cv2.imshow("Edged Map", accumEdged )

# detect, sort(areawise) contours
cnts= cv2.findContours(accumEdged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts= imutils.grab_contours(cnts)
cnts = sorted( cnts, key= cv2.contourArea, reverse= True)[:5]
orig = image.copy()

# call function draw_contours to draw contours sorted according tp area
for (i,c) in enumerate(cnts):
    orig= draw_contour(orig, c, i)

cv2.imshow('Sorted according to area', orig)

# call function sort_contours to sort according to method
(cnts, boundingBoxes)= sort_contours(cnts, method= 'top-to-bottom')


for (i,c) in enumerate(cnts):
    draw_contour(image, c, i)

#display image with sorted contours
cv2.imshow('Sorted according to method', image)
cv2.waitKey(0)